In [ ]:
from __future__ import annotations

%cd ../

In [2]:
import requests

base_url = "https://rest.uniprot.org/uniprotkb/search?query=reviewed:true+AND+organism_id:9606&format=tsv"
cols = [
    "accession",  # Entry
    "id",  # Entry Name
    "protein_name",  # Protein names
    "gene_names",  # Gene Names
    "length",  # Length
    "cc_pharmaceutical",  # Pharmaceutical use
    "annotation_score",  # Annotation
    "go",  # Gene Ontology (GO)
    "go_f",  # GO molecular function
    "go_p",  # GO biological process
    "mass",  # Mass
    "sequence",  # Sequence
    "cc_catalytic_activity",  # Catalytic activity
    "cc_interaction",  # Interacts with
    "cc_developmental_stage",  # Developmental stage
    "cc_disease",  # Involvement in disease
    "ft_mutagen",  # Mutagenesis
    "cc_biotechnology",  # Biotechnological use
    "cc_allergen",  # Allergenic Properties
    "cc_subcellular_location",  # Subcellular location
    "keyword",  # Keywords
]
with_return = base_url + "&size=200&fields=" + ",".join(cols)

In [ ]:
import re
import time

page_number = 1
results_written = 0
max_page = 1e5
next_url = with_return

with open("uniprot_results.tsv", "w", encoding="utf-8") as f:
    while next_url and page_number < max_page:
        try:
            # Make the request with headers to get Link header
            response = requests.get(next_url)
            response.raise_for_status()

            # Get the response content
            content = response.text

            # Write headers only for the first page
            if page_number == 1:
                f.write(content)
            else:
                # For subsequent pages, skip the header
                lines = content.split("\n")
                if len(lines) > 1:  # Make sure there's data
                    f.write("\n".join(lines[1:]))

            # Update results count
            results_written += len(content.split("\n")) - 2  # Subtract 1 for header

            # Get the Link header
            link_header = response.headers.get("Link", "")

            # Extract next URL if it exists
            next_url = None
            if 'rel="next"' in link_header:
                # Use regex to extract the URL between < and >
                match = re.search(r'<([^>]+)>;\s*rel="next"', link_header)
                if match:
                    next_url = match.group(1)

            if results_written % 1000 == 0:
                print(f"Processed page {page_number}, Total results: {results_written}")

            # Add a small delay to avoid hitting rate limits
            time.sleep(1)
            page_number += 1

        except requests.exceptions.RequestException as e:
            print(f"Error occurred on page {page_number}: {e}")
            # Implement exponential backoff
            retry_delay = min(300, 2 ** (page_number - 1))  # Cap at 5 minutes
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            continue

        except Exception as e:
            print(f"Unexpected error occurred: {e}")
            break

In [ ]:
import pandas as pd

df = pd.read_csv("uniprot_results.tsv", sep="\t")
df.head(2)

In [ ]:
len(df)

# Process catalytic activity feature

In [6]:
import re


def extract_catalytic_activity_features(activity_string):
    # Compile all patterns
    if activity_string is None or str(activity_string) == "nan":
        return {}
    patterns = {
        "reaction": re.compile(r"Reaction=(.+?);"),
        "ec": re.compile(r"EC=(\d+\.\d+\.\d+\.\d+);"),
    }

    # Extract all features
    features = {}
    for key, pattern in patterns.items():
        match = pattern.search(activity_string)
        if match:
            features[key] = match.group(1)

    return features

In [7]:
def clean_biotechnology_text(text):
    if str(text) == "nan":
        return "Not Available"
    # Remove "BIOTECHNOLOGY: " prefix and trailing "{ECO:...}"
    pattern = r"^BIOTECHNOLOGY:\s*(.*?)\s*\{ECO:.*\}\.?$"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text

In [8]:
def clean_subcellular_location(text):
    if str(text) == "nan":
        return "Not Available"
    text = re.sub(r"^SUBCELLULAR LOCATION:\s*", "", text)
    text = re.sub(r"\{[^}]*\}", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\.\s*\.", ".", text)
    text = text.replace(".", ";").replace(" ;", ";").strip().rstrip(".")
    return text

In [9]:
def clean_go_terms(text):
    if str(text) == "nan":
        return "Not Available"
    cleaned_text = re.sub(r"\s*\[GO:\d+\]", "", text)
    terms = [term.strip() for term in cleaned_text.split(";")]
    return "; ".join(terms)

In [10]:
all_entry_names = set(df["Entry"])


def clean_interaction_text(text):
    if str(text) == "nan":
        return "Not Available"
    process_s = "; ".join([s for s in text.split("; ") if s in all_entry_names])
    return process_s.rstrip("; ")

In [11]:
def clean_development_text(text):
    if str(text) == "nan":
        return "Not Available"
    pattern = r"DEVELOPMENTAL STAGE:\s*(.*?)\s*\{ECO:.*\}\."
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    return text

In [12]:
def parse_mutagen_entries(text):
    if text is None or str(text) == "nan":
        return []
    entries = re.split(r"MUTAGEN\s+", text)
    entries = [entry.strip() for entry in entries if entry.strip()]

    parsed_entries = []
    for entry in entries:
        position_match = re.match(r"(\d+\.\.\d+);", entry)
        note_match = re.search(r'/note="([^"]+)"', entry)

        if position_match and note_match:
            position = position_match.group(1)
            note = note_match.group(1)
            parsed_entries.append({"mutagen": position, "note": note})

    return parsed_entries

In [13]:
df["Catalytic activity"] = df["Catalytic activity"].apply(
    extract_catalytic_activity_features
)
df["Gene Ontology (biological process)"] = df[
    "Gene Ontology (biological process)"
].replace(float("nan"), "Not Available")
df["Biotechnological use"] = df["Biotechnological use"].apply(clean_biotechnology_text)
df["Involvement in disease"] = df["Involvement in disease"].replace(
    float("nan"), "Not Available"
)
df["Allergenic Properties"] = df["Allergenic Properties"].replace(
    float("nan"), "Not Available"
)
df["Pharmaceutical use"] = df["Pharmaceutical use"].replace(
    float("nan"), "Not Available"
)
df["Subcellular location [CC]"] = df["Subcellular location [CC]"].apply(
    clean_subcellular_location
)
df["Gene Ontology (GO)"] = df["Gene Ontology (GO)"].apply(clean_go_terms)
df["Gene Ontology (molecular function)"] = df[
    "Gene Ontology (molecular function)"
].apply(clean_go_terms)
df["Interacts with"] = df["Interacts with"].apply(clean_interaction_text)
df["Developmental stage"] = df["Developmental stage"].apply(clean_development_text)
df["Mutagenesis"] = df["Mutagenesis"].apply(parse_mutagen_entries)
df = df[df["Annotation"] >= 4].drop(columns=["Annotation"])
df = df[df["Sequence"].str.len() <= 1022]

In [14]:
df.to_csv("raw_data/new_uniprot_processed.tsv", sep="\t", index=False)

In [ ]:
len(df)

In [ ]:
%cd ../

import pandas as pd

df = pd.read_csv("raw_data/new_uniprot_processed.tsv", sep="\t")
df.head(2)

In [2]:
names = df["Entry"].tolist()
sequences = df["Sequence"].tolist()

In [ ]:
from nl2prot.data.utils import write_fasta

write_fasta(names, sequences, "raw_data/uniprot.fasta")